In [12]:
# CSV-Based Algorithmic Trading Backtester
# Comprehensive Test Suite and Performance Analysis

import datetime
import unittest
from typing import List, Dict
import sys
import os

# Add current directory to path so we can import our modules
sys.path.append('.')

# Import our backtester modules
from models import MarketDataPoint, Order, Position, OrderError, ExecutionError
from data_loader import load_market_data, generate_sample_data
from strategies import MovingAverageCrossoverStrategy, MomentumStrategy, BuyAndHoldStrategy
from engine import BacktestEngine
from reporting import generate_markdown_report, print_summary_stats

print("All modules imported successfully!")
print("Starting comprehensive backtester analysis...")

All modules imported successfully!
Starting comprehensive backtester analysis...


## 1. Unit Tests - Testing Core Functionality

### Testing Immutable vs Mutable Behavior
We'll test that MarketDataPoint is truly immutable while Order is mutable as required.

In [13]:
# Test 1: CSV Parsing into Frozen Dataclass
def test_market_data_immutability():
    """Test that MarketDataPoint is immutable."""
    
    print("=== Testing MarketDataPoint Immutability ===")
    
    # Create a MarketDataPoint
    timestamp = datetime.datetime(2024, 1, 1, 9, 30, 0)
    data_point = MarketDataPoint(timestamp=timestamp, symbol="AAPL", price=150.0)
    
    print(f"Created MarketDataPoint: {data_point}")
    
    # Try to modify the price (should fail)
    try:
        data_point.price = 160.0
        print("ERROR: MarketDataPoint price was modified (should be immutable!)")
        return False
    except AttributeError as e:
        print(f"SUCCESS: MarketDataPoint is immutable - {e}")
    
    # Try to modify the symbol (should fail)
    try:
        data_point.symbol = "GOOGL"
        print("ERROR: MarketDataPoint symbol was modified (should be immutable!)")
        return False
    except AttributeError as e:
        print(f"SUCCESS: MarketDataPoint symbol is immutable - {e}")
    
    return True

# Run the test
test_market_data_immutability()

=== Testing MarketDataPoint Immutability ===
Created MarketDataPoint: MarketDataPoint(timestamp=datetime.datetime(2024, 1, 1, 9, 30), symbol='AAPL', price=150.0)
SUCCESS: MarketDataPoint is immutable - cannot assign to field 'price'
SUCCESS: MarketDataPoint symbol is immutable - cannot assign to field 'symbol'


True

In [14]:
# Test 2: Mutable Behavior of Order
def test_order_mutability():
    """Test that Order class is mutable as required."""
    
    print("\n=== Testing Order Mutability ===")
    
    # Create an Order
    order = Order(symbol="AAPL", quantity=100, price=150.0)
    print(f"Created Order: {order}")
    
    # Test that we can update the order status
    original_status = order.status
    print(f"Original status: {original_status}")
    
    try:
        order.update_status('filled')
        print(f"SUCCESS: Order status updated to: {order.status}")
    except Exception as e:
        print(f"ERROR: Failed to update order status - {e}")
        return False
    
    # Test that we can modify other mutable attributes
    try:
        order.quantity = 200
        order.price = 155.0
        print(f"SUCCESS: Order quantity and price updated: {order}")
    except Exception as e:
        print(f"ERROR: Failed to update order attributes - {e}")
        return False
    
    return True

# Run the test
test_order_mutability()


=== Testing Order Mutability ===
Created Order: Order(symbol='AAPL', quantity=100, price=150.00, status='pending')
Original status: pending
SUCCESS: Order status updated to: filled
SUCCESS: Order quantity and price updated: Order(symbol='AAPL', quantity=200, price=155.00, status='filled')


True

In [15]:
# Test 3: Exception Raising and Handling
def test_exception_handling():
    """Test custom exceptions and error handling."""
    
    print("\n=== Testing Exception Handling ===")
    
    # Test OrderError for invalid parameters
    print("Testing OrderError for invalid order parameters...")
    
    # Test zero quantity
    try:
        Order(symbol="AAPL", quantity=0, price=150.0)
        print("ERROR: OrderError not raised for zero quantity")
        return False
    except OrderError as e:
        print(f"SUCCESS: OrderError raised for zero quantity - {e}")
    
    # Test negative price
    try:
        Order(symbol="AAPL", quantity=100, price=-150.0)
        print("ERROR: OrderError not raised for negative price")
        return False
    except OrderError as e:
        print(f"SUCCESS: OrderError raised for negative price - {e}")
    
    # Test invalid status
    try:
        Order(symbol="AAPL", quantity=100, price=150.0, status="invalid_status")
        print("ERROR: OrderError not raised for invalid status")
        return False
    except OrderError as e:
        print(f"SUCCESS: OrderError raised for invalid status - {e}")
    
    # Test ExecutionError simulation
    print("\nTesting ExecutionError simulation...")
    try:
        raise ExecutionError("Simulated market connectivity issue")
    except ExecutionError as e:
        print(f"SUCCESS: ExecutionError caught and handled - {e}")
    
    return True

# Run the test
test_exception_handling()


=== Testing Exception Handling ===
Testing OrderError for invalid order parameters...
SUCCESS: OrderError raised for zero quantity - Order quantity cannot be zero
SUCCESS: OrderError raised for negative price - Order price must be positive
SUCCESS: OrderError raised for invalid status - Invalid order status: invalid_status

Testing ExecutionError simulation...
SUCCESS: ExecutionError caught and handled - Simulated market connectivity issue


True

## 2. Data Generation and Loading

Let's generate sample market data and test our data loading functionality.

In [16]:
# Generate sample market data
print("Generating sample market data...")

# Check if data file already exists
data_file = "market_data.csv"

if not os.path.exists(data_file):
    generate_sample_data(data_file, num_ticks=500)
    print(f"Generated {data_file} with 500 data points")
else:
    print(f"{data_file} already exists")

# Load and inspect the data
print(f"\nLoading market data from {data_file}...")
market_data = load_market_data(data_file)

print(f"\nMarket Data Summary:")
print(f"  Total data points: {len(market_data)}")
print(f"  Symbol: {market_data[0].symbol}")
print(f"  Time range: {market_data[0].timestamp} to {market_data[-1].timestamp}")
print(f"  Price range: ${min(d.price for d in market_data):.2f} to ${max(d.price for d in market_data):.2f}")

# Show first few data points
print(f"\nFirst 5 data points:")
for i, data_point in enumerate(market_data[:5]):
    print(f"  {i+1}. {data_point}")

print(f"\nLast 5 data points:")
for i, data_point in enumerate(market_data[-5:]):
    print(f"  {len(market_data)-4+i}. {data_point}")

# Verify data points are MarketDataPoint instances
print(f"\nData validation:")
print(f"  All objects are MarketDataPoint instances: {all(isinstance(d, MarketDataPoint) for d in market_data)}")
print(f"  Data is sorted by timestamp: {market_data == sorted(market_data, key=lambda x: x.timestamp)}")

Generating sample market data...
market_data.csv already exists

Loading market data from market_data.csv...
Loaded 500 market data points from market_data.csv

Market Data Summary:
  Total data points: 500
  Symbol: AAPL
  Time range: 2024-01-01 09:30:00 to 2024-01-01 17:49:00
  Price range: $80.85 to $167.46

First 5 data points:
  1. MarketDataPoint(timestamp=datetime.datetime(2024, 1, 1, 9, 30), symbol='AAPL', price=144.52)
  2. MarketDataPoint(timestamp=datetime.datetime(2024, 1, 1, 9, 31), symbol='AAPL', price=150.16)
  3. MarketDataPoint(timestamp=datetime.datetime(2024, 1, 1, 9, 32), symbol='AAPL', price=150.17)
  4. MarketDataPoint(timestamp=datetime.datetime(2024, 1, 1, 9, 33), symbol='AAPL', price=149.63)
  5. MarketDataPoint(timestamp=datetime.datetime(2024, 1, 1, 9, 34), symbol='AAPL', price=151.3)

Last 5 data points:
  496. MarketDataPoint(timestamp=datetime.datetime(2024, 1, 1, 17, 45), symbol='AAPL', price=96.76)
  497. MarketDataPoint(timestamp=datetime.datetime(2024,

## 3. Strategy Testing and Demonstration

Let's test our trading strategies individually before running the full backtest.

In [17]:
# Test strategies individually
def test_strategies(market_data_sample):
    """Test each strategy with a sample of market data."""
    
    print("=== Testing Trading Strategies ===")
    
    # Take first 50 data points for strategy testing
    test_data = market_data_sample[:50]
    
    # Test Moving Average Strategy
    print("\n1. Testing Moving Average Crossover Strategy:")
    ma_strategy = MovingAverageCrossoverStrategy(short_window=5, long_window=10, position_size=100)
    ma_signals = []
    
    for i, tick in enumerate(test_data):
        signals = ma_strategy.generate_signals(tick)
        if signals:
            ma_signals.extend(signals)
            print(f"   Tick {i+1}: Generated {len(signals)} signal(s) at price ${tick.price:.2f}")
            for signal in signals:
                print(f"     -> {signal}")
    
    print(f"   Total MA signals generated: {len(ma_signals)}")
    
    # Test Momentum Strategy
    print("\n2. Testing Momentum Strategy:")
    momentum_strategy = MomentumStrategy(lookback_period=3, momentum_threshold=0.01, position_size=50)
    momentum_signals = []
    
    for i, tick in enumerate(test_data):
        signals = momentum_strategy.generate_signals(tick)
        if signals:
            momentum_signals.extend(signals)
            print(f"   Tick {i+1}: Generated {len(signals)} signal(s) at price ${tick.price:.2f}")
            for signal in signals:
                print(f"     -> {signal}")
    
    print(f"   Total Momentum signals generated: {len(momentum_signals)}")
    
    # Test Buy and Hold Strategy
    print("\n3. Testing Buy and Hold Strategy:")
    buy_hold_strategy = BuyAndHoldStrategy(position_size=200)
    buy_hold_signals = []
    
    for i, tick in enumerate(test_data):
        signals = buy_hold_strategy.generate_signals(tick)
        if signals:
            buy_hold_signals.extend(signals)
            print(f"   Tick {i+1}: Generated {len(signals)} signal(s) at price ${tick.price:.2f}")
            for signal in signals:
                print(f"     -> {signal}")
    
    print(f"   Total Buy-and-Hold signals generated: {len(buy_hold_signals)}")
    
    return ma_signals, momentum_signals, buy_hold_signals

# Run strategy tests
ma_signals, momentum_signals, buy_hold_signals = test_strategies(market_data)

=== Testing Trading Strategies ===

1. Testing Moving Average Crossover Strategy:
   Tick 13: Generated 1 signal(s) at price $157.14
     -> ('sell', 'AAPL', 100, 157.14)
   Tick 14: Generated 2 signal(s) at price $157.03
     -> ('buy', 'AAPL', 100, 157.03)
     -> ('buy', 'AAPL', 100, 157.03)
   Tick 19: Generated 2 signal(s) at price $150.65
     -> ('sell', 'AAPL', 100, 150.65)
     -> ('sell', 'AAPL', 100, 150.65)
   Tick 22: Generated 2 signal(s) at price $160.12
     -> ('buy', 'AAPL', 100, 160.12)
     -> ('buy', 'AAPL', 100, 160.12)
   Tick 32: Generated 2 signal(s) at price $151.54
     -> ('sell', 'AAPL', 100, 151.54)
     -> ('sell', 'AAPL', 100, 151.54)
   Tick 47: Generated 2 signal(s) at price $152.43
     -> ('buy', 'AAPL', 100, 152.43)
     -> ('buy', 'AAPL', 100, 152.43)
   Tick 48: Generated 2 signal(s) at price $145.14
     -> ('sell', 'AAPL', 100, 145.14)
     -> ('sell', 'AAPL', 100, 145.14)
   Total MA signals generated: 13

2. Testing Momentum Strategy:
   Tick 

## 4. Full Backtesting Engine Execution

Now let's run the complete backtesting engine with all strategies.

In [18]:
# Initialize and run the backtesting engine
print("=== Running Full Backtest ===")

# Configuration
initial_cash = 100000.0
commission = 1.0

# Initialize engine
engine = BacktestEngine(initial_cash=initial_cash, commission=commission)

# Add strategies
print("Adding strategies to engine...")

# Moving Average Crossover Strategy
ma_strategy = MovingAverageCrossoverStrategy(
    short_window=10, 
    long_window=20, 
    position_size=100
)
engine.add_strategy(ma_strategy)
print("  ✓ Added Moving Average Crossover Strategy")

# Momentum Strategy
momentum_strategy = MomentumStrategy(
    lookback_period=5,
    momentum_threshold=0.02,
    position_size=50
)
engine.add_strategy(momentum_strategy)
print("  ✓ Added Momentum Strategy")

# Buy and Hold Strategy
buy_hold_strategy = BuyAndHoldStrategy(position_size=200)
engine.add_strategy(buy_hold_strategy)
print("  ✓ Added Buy and Hold Strategy")

print(f"\nStarting backtest with:")
print(f"  • {len(market_data)} market data points")
print(f"  • ${initial_cash:,.2f} initial cash")
print(f"  • ${commission:.2f} commission per trade")
print(f"  • {len(engine.strategies)} active strategies")

# Run the backtest
results = engine.run_backtest(market_data)

print("\n" + "="*60)
print("BACKTEST COMPLETED!")
print("="*60)

=== Running Full Backtest ===
Adding strategies to engine...
  ✓ Added Moving Average Crossover Strategy
  ✓ Added Momentum Strategy
  ✓ Added Buy and Hold Strategy

Starting backtest with:
  • 500 market data points
  • $100,000.00 initial cash
  • $1.00 commission per trade
  • 3 active strategies
Starting backtest with 500 data points...
Initial cash: $100,000.00
Active strategies: 3
Processed 100/500 ticks. Portfolio value: $98,642.00
Processed 200/500 ticks. Portfolio value: $92,154.50
Processed 300/500 ticks. Portfolio value: $91,105.00
Processed 400/500 ticks. Portfolio value: $89,709.00
Processed 500/500 ticks. Portfolio value: $90,496.00
Backtest completed!
Final portfolio value: $90,496.00
Total return: -9.50%
Total trades: 183
Failed orders: 8
Errors encountered: 8

BACKTEST COMPLETED!


## 5. Performance Analysis and Results

Let's analyze the results in detail.

In [19]:
# Display comprehensive results
print_summary_stats(results)

# Detailed performance breakdown
print(f"\n{'='*60}")
print("DETAILED PERFORMANCE BREAKDOWN")
print(f"{'='*60}")

print(f"\nPortfolio Performance:")
print(f"  Initial Value:     ${results['initial_value']:>12,.2f}")
print(f"  Final Value:       ${results['final_value']:>12,.2f}")
print(f"  Total P&L:         ${results['final_value'] - results['initial_value']:>12,.2f}")
print(f"  Total Return:      {results['total_return']*100:>13.2f}%")

print(f"\nRisk Metrics:")
print(f"  Sharpe Ratio:      {results['sharpe_ratio']:>13.3f}")
print(f"  Max Drawdown:      {results['max_drawdown']*100:>13.2f}%")

if results['returns']:
    volatility = (sum((r - sum(results['returns'])/len(results['returns']))**2 
                     for r in results['returns']) / len(results['returns'])) ** 0.5 * (252**0.5) * 100
    print(f"  Annualized Vol:    {volatility:>13.2f}%")

print(f"\nTrading Activity:")
print(f"  Total Orders:      {results['total_trades']:>13,}")
print(f"  Successful:        {results['successful_trades']:>13,}")
print(f"  Failed:            {results['failed_trades']:>13,}")
print(f"  Success Rate:      {(results['successful_trades']/max(results['total_trades'],1)*100):>13.1f}%")

print(f"\nError Handling:")
print(f"  Total Errors:      {results['total_errors']:>13,}")

if results['positions']:
    print(f"\nFinal Positions:")
    for symbol, position in results['positions'].items():
        if position.quantity != 0:
            print(f"  {symbol:>6}: {position.quantity:>8} shares @ ${position.avg_price:>7.2f}")
            print(f"         Unrealized P&L: ${position.unrealized_pnl:>8.2f}")

# Display recent trades
if results['trades']:
    print(f"\nRecent Trades (Last 10):")
    for i, trade in enumerate(results['trades'][-10:], 1):
        action_symbol = "BUY" if trade['action'] == 'buy' else "SELL"
        print(f"  {i:2}. {action_symbol} {trade['quantity']:>3} {trade['symbol']} @ ${trade['price']:>7.2f}")

# Display error summary if any
if results['errors']:
    print(f"\nError Summary (First 5):")
    for i, error in enumerate(results['errors'][:5], 1):
        timestamp = error['timestamp'].strftime('%H:%M:%S')
        print(f"  {i}. [{timestamp}] {error['error']}")
    
    if len(results['errors']) > 5:
        print(f"     ... and {len(results['errors']) - 5} more errors")


BACKTEST SUMMARY
Initial Value:    $     100,000.00
Final Value:      $      90,496.00
Total Return:              -9.50%
Sharpe Ratio:              -0.691
Max Drawdown:              15.18%
Total Trades:                 183
Success Rate:              100.0%
Errors:                         8

DETAILED PERFORMANCE BREAKDOWN

Portfolio Performance:
  Initial Value:     $  100,000.00
  Final Value:       $   90,496.00
  Total P&L:         $   -9,504.00
  Total Return:              -9.50%

Risk Metrics:
  Sharpe Ratio:             -0.691
  Max Drawdown:              15.18%
  Annualized Vol:             6.95%

Trading Activity:
  Total Orders:                183
  Successful:                  183
  Failed:                        8
  Success Rate:              100.0%

Error Handling:
  Total Errors:                  8

Final Positions:
    AAPL:      250 shares @ $  90.69
         Unrealized P&L: $  614.44

Recent Trades (Last 10):
   1. SELL  50 AAPL @ $  84.89
   2. SELL  50 AAPL @ $  84.89

In [20]:
# Create visual portfolio equity curve using basic ASCII art
def create_ascii_equity_curve(portfolio_values, width=80, height=20):
    """Create a simple ASCII chart of portfolio values."""
    
    if not portfolio_values or len(portfolio_values) < 2:
        return "No data available for chart"
    
    lines = []
    lines.append("Portfolio Equity Curve (ASCII Visualization)")
    lines.append("=" * width)
    
    # Normalize values to chart dimensions
    min_val = min(portfolio_values)
    max_val = max(portfolio_values)
    
    if max_val == min_val:
        lines.append("Portfolio value remained constant")
        return "\n".join(lines)
    
    # Scale values to chart height
    scaled_values = []
    for val in portfolio_values:
        scaled = int((val - min_val) / (max_val - min_val) * (height - 1))
        scaled_values.append(scaled)
    
    # Create chart grid
    chart = [[' ' for _ in range(width)] for _ in range(height)]
    
    # Plot data points
    for i, scaled_val in enumerate(scaled_values):
        if i < width:
            y = height - 1 - scaled_val  # Invert Y axis
            chart[y][i] = '*'
    
    # Convert chart to strings
    for row in chart:
        lines.append(''.join(row))
    
    # Add scale information
    lines.append("=" * width)
    lines.append(f"Min: ${min_val:>8,.0f} | Max: ${max_val:>8,.0f} | Data Points: {len(portfolio_values)}")
    
    return "\n".join(lines)

# Display the equity curve
print("\n" + create_ascii_equity_curve(results['portfolio_values']))


Portfolio Equity Curve (ASCII Visualization)
                         *                                                      
                       ** **                                                    
                            **                                                  
       *    *        **       *                                         *       
 ****** **   **     *          **  ** **  **  *                        * *  ** *
          **     ***             **  *  **  ** **                         **  * 
*               *                                **********************         
               *                                                                
                                                                                
                                                                                
                                                                                
                                                               

## 6. Generate Comprehensive Performance Report

Let's generate the full Markdown report with all metrics, tables, and analysis.

In [21]:
# Generate the comprehensive Markdown report
report_filename = "performance_report.md"
generate_markdown_report(results, report_filename)

print(f"✅ Comprehensive performance report generated: {report_filename}")

# Display a preview of the report content
print(f"\n{'='*60}")
print("REPORT PREVIEW")
print(f"{'='*60}")

try:
    with open(report_filename, 'r', encoding='utf-8') as f:
        content = f.read()
        # Show first 1000 characters of the report
        preview = content[:1000]
        if len(content) > 1000:
            preview += "\n\n... (report continues) ..."
        print(preview)
        
    print(f"\nFull report available at: {report_filename}")
    print(f"Report size: {len(content):,} characters")
    
except FileNotFoundError:
    print(f"Error: Could not read the generated report file.")
except Exception as e:
    print(f"Error reading report: {e}")

Performance report generated: performance_report.md
✅ Comprehensive performance report generated: performance_report.md

REPORT PREVIEW
# Algorithmic Trading Backtest Performance Report

**Report Generated:** 2026-01-18 17:55:20

## Executive Summary

The strategy generated negative returns of **-9.50%** over the backtest period. 
The strategy achieved a Sharpe ratio of **-0.691**, indicating poor risk-adjusted returns. 
Maximum drawdown was **15.18%**, representing the largest peak-to-trough decline.

## Key Performance Metrics

| Metric | Value |
|--------|-------|
| Initial Portfolio Value | $100,000.00 |
| Final Portfolio Value | $90,496.00 |
| Total Return | -9.50% |
| Sharpe Ratio | -0.691 |
| Maximum Drawdown | 15.18% |
| Total Trades | 183 |
| Successful Trades | 183 |
| Failed Trades | 8 |
| Trade Success Rate | 100.0% |

## Portfolio Evolution

### Equity Curve (ASCII Visualization)

```
                       ******                               
       *             **     

## 7. Summary and Key Learning Demonstrations

Let's demonstrate that we've successfully implemented all the required components and learning objectives.

In [22]:
# Comprehensive Learning Objectives Verification
print("="*80)
print("LEARNING OBJECTIVES VERIFICATION")
print("="*80)

learning_objectives = [
    "1. Parse CSV data into immutable dataclass instances",
    "2. Distinguish and use mutable classes for order management", 
    "3. Build an abstract Strategy interface with concrete subclasses",
    "4. Manage time-series data and portfolio state using lists and dictionaries",
    "5. Define custom exceptions and handle errors without stopping the backtest",
    "6. Generate a Markdown report summarizing key performance metrics"
]

print("\nSUCCESSFULLY DEMONSTRATED:")

print(f"\n{learning_objectives[0]}:")
print(f"   • Created frozen @dataclass MarketDataPoint")
print(f"   • Loaded {len(market_data)} CSV records into MarketDataPoint instances")
print(f"   • Verified immutability: Cannot modify MarketDataPoint.price after creation")

print(f"\n{learning_objectives[1]}:")
print(f"   • Created mutable Order class with updateable status")
print(f"   • Demonstrated Order.update_status() functionality")
print(f"   • Order attributes can be modified after instantiation")

print(f"\n{learning_objectives[2]}:")
print(f"   • Created abstract Strategy ABC with @abstractmethod generate_signals")
print(f"   • Implemented 3 concrete strategy subclasses:")
print(f"     - MovingAverageCrossoverStrategy")
print(f"     - MomentumStrategy") 
print(f"     - BuyAndHoldStrategy")

print(f"\n{learning_objectives[3]}:")
print(f"   • Market data managed in list: {len(market_data)} chronological data points")
print(f"   • Portfolio positions tracked in dictionary by symbol")
print(f"   • Portfolio values stored as time series list: {len(results['portfolio_values'])} points")
print(f"   • Trading signals collected in lists before order conversion")

print(f"\n{learning_objectives[4]}:")
print(f"   • Defined custom OrderError and ExecutionError exceptions")
print(f"   • Handled {results['total_errors']} errors during backtest without stopping")
print(f"   • Failed {results['failed_trades']} orders gracefully with error logging")
print(f"   • Continued processing despite individual order failures")

print(f"\n{learning_objectives[5]}:")
print(f"   • Generated comprehensive Markdown report: {report_filename}")
print(f"   • Report includes performance metrics tables, equity curve, and insights")
print(f"   • Calculated Sharpe ratio: {results['sharpe_ratio']:.3f}")
print(f"   • Calculated maximum drawdown: {results['max_drawdown']*100:.2f}%")

print(f"\n" + "="*80)
print("KEY TECHNICAL ACHIEVEMENTS")
print("="*80)

print(f"\nDATA PROCESSING:")
print(f"   • CSV parsing with datetime handling and error resilience")
print(f"   • Type-safe immutable data structures using frozen dataclass")
print(f"   • Proper separation of data loading and business logic")

print(f"\nOBJECT-ORIENTED DESIGN:")
print(f"   • Abstract base class with inheritance hierarchy")
print(f"   • Encapsulation with private attributes (e.g., _prices, _window)")
print(f"   • Polymorphism through Strategy interface")

print(f"\nALGORITHM IMPLEMENTATION:")
print(f"   • Moving averages with crossover detection")
print(f"   • Momentum calculation with configurable thresholds")
print(f"   • Portfolio state management and P&L tracking")

print(f"\nERROR HANDLING & RESILIENCE:")
print(f"   • Custom exception hierarchy")
print(f"   • Graceful degradation under failures")
print(f"   • Comprehensive error logging and reporting")

print(f"\nREPORTING & ANALYTICS:")
print(f"   • Financial metrics calculation (Sharpe, drawdown)")
print(f"   • Markdown report generation with tables and charts")
print(f"   • Multi-format output (console, Markdown, structured data)")

print(f"\nFINAL PERFORMANCE SUMMARY:")
print(f"   • Portfolio Return: {results['total_return']*100:>8.2f}%")
print(f"   • Risk-Adjusted Return: {results['sharpe_ratio']:>8.3f}")
print(f"   • Trade Success Rate: {(results['successful_trades']/max(results['total_trades'],1)*100):>8.1f}%")
print(f"   • System Reliability: {((results['total_trades']-results['total_errors'])/max(results['total_trades'],1)*100):>8.1f}%")

print(f"\nALL LEARNING OBJECTIVES SUCCESSFULLY COMPLETED!")
print("="*80)

LEARNING OBJECTIVES VERIFICATION

SUCCESSFULLY DEMONSTRATED:

1. Parse CSV data into immutable dataclass instances:
   • Created frozen @dataclass MarketDataPoint
   • Loaded 500 CSV records into MarketDataPoint instances
   • Verified immutability: Cannot modify MarketDataPoint.price after creation

2. Distinguish and use mutable classes for order management:
   • Created mutable Order class with updateable status
   • Demonstrated Order.update_status() functionality
   • Order attributes can be modified after instantiation

3. Build an abstract Strategy interface with concrete subclasses:
   • Created abstract Strategy ABC with @abstractmethod generate_signals
   • Implemented 3 concrete strategy subclasses:
     - MovingAverageCrossoverStrategy
     - MomentumStrategy
     - BuyAndHoldStrategy

4. Manage time-series data and portfolio state using lists and dictionaries:
   • Market data managed in list: 500 chronological data points
   • Portfolio positions tracked in dictionary by 